In [13]:
# Cek Versi Tensorflow
import tensorflow as tf
print(tf.__version__)

2.4.1


In [14]:
# Download Dataset Katarak
! KAGGLE_CONFIG_DIR=/content/ kaggle datasets download jr2ngb/cataractdataset
! chmod 600 kaggle.json
! ls ~/.kaggle 2>/dev/null || mkdir ~/.kaggle
! mv kaggle.json ~/.kaggle
! kaggle datasets download jr2ngb/cataractdataset

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /content/. Or use the environment method.
chmod: cannot access 'kaggle.json': No such file or directory
kaggle.json
mv: cannot stat 'kaggle.json': No such file or directory
100% 3.34G/3.34G [00:38<00:00, 53.7MB/s]
100% 3.34G/3.34G [00:38<00:00, 93.7MB/s]


In [ ]:
# Proses Ekstrak Dataset
import zipfile
import os

with zipfile.ZipFile('cataractdataset.zip','r') as z:
  z.extractall('./')

os.listdir()

In [19]:
import shutil
dir_path = '/content/repository'

shutil.rmtree(dir_path)

In [21]:
base_dir = '/content/dataset'

file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(base_dir):
    for name in files:
        full_path.append(os.path.join(path, name)) 
        tag.append(path.split('/')[-1])        
        file_name.append(name)

In [22]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.85 and logs.get('val_accuracy')>0.85):
            print("\nAkurasi telah mencapai lebih dari 85%, proses dihentikan!")
            self.model.stop_training = True

In [ ]:
import pandas as pd
df = pd.DataFrame({"path":full_path,'file_name':file_name,"tag":tag})
df.groupby(['tag']).size()

In [24]:
from sklearn.model_selection import train_test_split

X = df['path']
y = df['tag']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=300)

df_tr = pd.DataFrame({'path':X_train, 'tag':y_train, 'set':'train'})
df_val = pd.DataFrame({'path':X_val, 'tag':y_val, 'set':'validation'})

print('train size', len(df_tr))
print('val size', len(df_val))

df_all = df_tr.append([df_val]).reset_index(drop=1)
print(df_all.groupby(['set','tag']).size(),'\n')

df_all.sample(3)

train size 540
val size 61
set         tag             
train       1_normal            268
            2_cataract           88
            2_glaucoma           93
            3_retina_disease     91
validation  1_normal             32
            2_cataract           12
            2_glaucoma            8
            3_retina_disease      9
dtype: int64 



,path,tag,set
322,/content/dataset/1_normal/NL_289.png,1_normal,train
227,/content/dataset/2_glaucoma/Glaucoma_004.png,2_glaucoma,train
479,/content/dataset/2_glaucoma/Glaucoma_019.png,2_glaucoma,train


In [25]:
import shutil
from tqdm.notebook import tqdm as tq
datasource_path = base_dir
dataset_path = "content/dataset/dataset/"

for index, row in tq(df_all.iterrows()):
    
    #detect filepath
    file_path = row['path']            
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['tag'],row['image'].split('.')[0])  
    
    #make folder destination dirs
    if os.path.exists(os.path.join(dataset_path,row['set'],row['tag'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['tag']))
    
    #define file dest
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['tag'],destination_file_name)
    
    #copy file from source to dest
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

In [28]:
#Image Augmentation
from keras.preprocessing.image import ImageDataGenerator
train_dir = "content/dataset/dataset/train"
val_dir = "content/dataset/dataset/validation"

train_datagen = ImageDataGenerator(
                  rescale=1./255,
                  rotation_range=40,
                  vertical_flip=True,
                  shear_range = 0.2,
                  zoom_range=0.2,
                  fill_mode = 'nearest')

val_datagen = ImageDataGenerator(
                  rescale=1./255,
                  rotation_range=40,
                  vertical_flip=True,
                  shear_range = 0.2,
                  zoom_range=0.2,
                  fill_mode = 'nearest')

train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(150,150),  
        batch_size=4,
        class_mode='categorical')
 
validation_generator = val_datagen.flow_from_directory(
        val_dir, # direktori data validasi
        target_size=(150, 150), 
        batch_size=4, 
        class_mode='categorical')

#Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

callbacks = myCallback()
history = model.fit(
    train_generator,
    steps_per_epoch = 100,
    epochs = 20,
    validation_data = validation_generator,
    validation_steps = 5,
    verbose =2,
    callbacks=[callbacks]
)

Found 540 images belonging to 4 classes.
Found 61 images belonging to 4 classes.
Epoch 1/20
100/100 - 54s - loss: 1.3075 - accuracy: 0.5000 - val_loss: 1.0132 - val_accuracy: 0.6500
Epoch 2/20
100/100 - 53s - loss: 1.2626 - accuracy: 0.5075 - val_loss: 1.1870 - val_accuracy: 0.5500
Epoch 3/20
100/100 - 53s - loss: 1.2600 - accuracy: 0.5000 - val_loss: 1.2887 - val_accuracy: 0.4500
Epoch 4/20
100/100 - 54s - loss: 1.2744 - accuracy: 0.4800 - val_loss: 1.2416 - val_accuracy: 0.5000
Epoch 5/20
100/100 - 58s - loss: 1.2457 - accuracy: 0.5075 - val_loss: 0.9658 - val_accuracy: 0.7500
Epoch 6/20
100/100 - 53s - loss: 1.2516 - accuracy: 0.5050 - val_loss: 1.3019 - val_accuracy: 0.4500
Epoch 7/20
100/100 - 53s - loss: 1.2758 - accuracy: 0.4775 - val_loss: 1.1005 - val_accuracy: 0.6500
Epoch 8/20
100/100 - 53s - loss: 1.2848 - accuracy: 0.4650 - val_loss: 1.3648 - val_accuracy: 0.3500
Epoch 9/20
100/100 - 53s - loss: 1.2611 - accuracy: 0.5000 - val_loss: 1.1474 - val_accuracy: 0.6000
Epoch 10/2